In [58]:
import pandas as pd
import wikipediaapi
import numpy as np

In [59]:
#Extracting mountains data from web_site 

In [60]:
df_mountains = pd.read_html('https://en.wikipedia.org/wiki/List_of_mountains_by_elevation')

In [61]:
#I think top 100 mountains is enought

In [62]:
df_mountains = pd.concat([df_mountains[0], df_mountains[1]]).iloc[:100, :]['Mountain']

In [63]:
df_mountains.shape

(100,)

In [64]:
#There is many tables with mountains sorted by category
#We need to concat it in one table and choose top 100, we need only names

In [65]:
df_mountains.head(5)

0    Mount Everest
1               K2
2    Kangchenjunga
3           Lhotse
4           Makalu
Name: Mountain, dtype: object

In [66]:
#And now we need to find sentence for all mountains. We can do it with wikipedia-api

In [67]:
def get_mountain_sentences(mountain_name, num_sentences=7):
    wiki_wiki = wikipediaapi.Wikipedia('YourCustomUserAgent/1.0', 'en')
    page_py = wiki_wiki.page(mountain_name)

    sentences = []
    if page_py.exists():
        text = page_py.text
        # sentences = [sentence.strip() for sentence in text.split('.')[:num_sentences] if mountain_name in sentence]
        for sent in text.split('.'):
            if mountain_name in sent and sent.strip().index(mountain_name) != 0 :
                sentences.append(sent.strip())
    if sentences:            
        return ''.join(sentences[0])
    else:
        return np.nan

In [68]:
# and sent.strip().index(mountain_name) != 0

In [69]:
mount_sentences = []

In [70]:
for mount in list(df_mountains):
    mount_sentences.append(get_mountain_sentences(mount))

In [71]:
len(mount_sentences)

100

In [72]:
sentences = pd.Series(mount_sentences)

In [73]:
sentences

0     Waugh's proposed name prevailed despite the ob...
1     " Of the five highest mountains in the world, ...
2     Its summit lies at 8,586 m (28,169 ft) in a se...
3     With Everest to the north and Nuptse to the we...
4     One of the eight-thousanders, Makalu is an iso...
                            ...                        
95    Kazimierza Sosnkowskiego\n\nTransportation\nHM...
96                                                  NaN
97    There are two known approaches to Mana Peak: o...
98                                                  NaN
99    See also\nList of mountains in Pakistan\nList ...
Length: 100, dtype: object

In [74]:
df_mountains = df_mountains.reset_index(drop=True)

In [75]:
#Create dataset

In [76]:
df_mountains = pd.concat([df_mountains, sentences.rename('Sentence')], axis=1)

In [77]:
df_mountains

,Mountain,Sentence
0,Mount Everest,Waugh's proposed name prevailed despite the ob...
1,K2,""" Of the five highest mountains in the world, ..."
2,Kangchenjunga,"Its summit lies at 8,586 m (28,169 ft) in a se..."
3,Lhotse,With Everest to the north and Nuptse to the we...
4,Makalu,"One of the eight-thousanders, Makalu is an iso..."
...,...,...
95,K6,Kazimierza Sosnkowskiego\n\nTransportation\nHM...
96,Kangpenqing,NaN
97,Mana Peak,There are two known approaches to Mana Peak: o...
98,Muztagh Tower,NaN


In [78]:
#Clean broken data

In [79]:
df_mountains.isna().sum()

Mountain     0
Sentence    23
dtype: int64

In [80]:
df_mountains.dropna(inplace=True)

In [81]:
df_mountains.shape

(77, 2)

In [82]:
df_mountains = df_mountains.reset_index(drop=True)
df_mountains = df_mountains.drop(index=[57, 74, 76])

In [83]:
#save to disk

In [84]:
df_mountains.to_csv('db.csv', index=False)